## Make a large DataFrame from all the brewerydb IPAs

In [1]:
import pandas as pd
import pickle
from glob import glob

In [2]:
bdb_ipas = glob('capstone_1/brewkettle/*.pkl')
bdb_ipas

['capstone_1/brewkettle/style_172_df.pkl',
 'capstone_1/brewkettle/style_26_df.pkl',
 'capstone_1/brewkettle/style_171_df.pkl',
 'capstone_1/brewkettle/style_31_df.pkl',
 'capstone_1/brewkettle/style_173_df.pkl',
 'capstone_1/brewkettle/bDb_style_30.pkl',
 'capstone_1/brewkettle/style_2_df.pkl']

In [3]:
# Decided against using style 2, which is English IPA
bdb_ipas.pop()

'capstone_1/brewkettle/style_2_df.pkl'

In [4]:
# seems like style 30 was structured differently, so start with it
with open('capstone_1/brewkettle/bDb_style_30.pkl', 'rb') as f:
    s30 = pickle.load(f)
print(type(s30), len(s30))

<class 'pandas.core.frame.DataFrame'> 9650


In [7]:
s30.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9650 entries, 0 to 9649
Data columns (total 32 columns):
index                        9650 non-null int64
abv                          8886 non-null object
available                    2568 non-null object
availableId                  2568 non-null float64
beerVariation                52 non-null object
beerVariationId              52 non-null object
breweries                    9650 non-null object
createDate                   9650 non-null object
description                  6697 non-null object
foodPairings                 350 non-null object
glass                        1726 non-null object
glasswareId                  1726 non-null float64
ibu                          4842 non-null object
id                           9650 non-null object
ingredients                  290 non-null object
isOrganic                    9650 non-null object
isRetired                    9650 non-null object
labels                       3542 non-null objec

In [20]:
# Hand pick the features
keepcols = ['id', 'breweries', 'name','abv', 'description',
            'ibu', 'ingredients', 'socialAccounts', 'styleId']

In [79]:
s30 = s30[keepcols]

In [126]:
s30.ingredients[17]

{'hops': [{'category': 'hop',
   'categoryDisplay': 'Hops',
   'createDate': '2013-06-24 16:07:29',
   'id': 30,
   'name': 'Columbus'},
  {'alphaAcidMin': 3.3,
   'betaAcidMax': 9.5,
   'betaAcidMin': 5.4,
   'caryophylleneMax': 10,
   'caryophylleneMin': 6.5,
   'category': 'hop',
   'categoryDisplay': 'Hops',
   'cohumuloneMax': 13,
   'cohumuloneMin': 11,
   'country': {'createDate': '2012-01-03 02:41:33',
    'displayName': 'United States',
    'isoCode': 'US',
    'isoThree': 'USA',
    'name': 'UNITED STATES',
    'numberCode': 840},
   'countryOfOrigin': 'US',
   'createDate': '2013-06-24 16:07:31',
   'description': 'Released in 2000 from Washington State University, supposed to be a balanced bittering/aroma hop. Said to be a more citrusy version of Willamette.',
   'farneseneMax': 0.3,
   'farneseneMin': 0,
   'humuleneMax': 36,
   'humuleneMin': 24,
   'id': 51,
   'myrceneMax': 62,
   'myrceneMin': 33,
   'name': 'Glacier',
   'updateDate': '2013-06-24 16:10:37'},
  {'alpha

In [80]:
# check style 31 structure
with open('capstone_1/brewkettle/style_31_df.pkl', 'rb') as f:
    s31 = pickle.load(f)
print(type(s31), len(s31))

<class 'pandas.core.frame.DataFrame'> 83


In [112]:
print(s31.shape, s31.columns)

(83, 5) Index(['currentPage', 'data', 'numberOfPages', 'status', 'totalResults'], dtype='object')


There are 83 batches, and only 1 column that matters here, 'data',
   which is a list of API responses, each list item being 50 results

In [95]:
len(s31.data[0])

50

In [110]:
# First flatten out each list and then concat them all
concat31 = pd.concat([pd.io.json.json_normalize(data) for data in s31.data], axis=0, ignore_index=True)

In [111]:
concat31.shape

(4111, 92)

In [113]:
# repeat for styles 26, 171, 172, and 173
def flatten_dicts(style):
    with open(f'capstone_1/brewkettle/style_{style}_df.pkl', 'rb') as f:
        nest = pickle.load(f)
    return pd.concat([pd.io.json.json_normalize(data) for data in nest.data],
                        axis=0, ignore_index=True)

In [114]:
concat26 = flatten_dicts(26)
concat171 = flatten_dicts(171)
concat172 = flatten_dicts(172)
concat173 = flatten_dicts(173)

In [115]:
concat26.columns

Index(['abv', 'available.description', 'available.id', 'available.name',
       'availableId', 'breweries', 'createDate', 'description', 'foodPairings',
       'glass.createDate', 'glass.id', 'glass.name', 'glasswareId', 'ibu',
       'id', 'ingredients.hops', 'ingredients.malt', 'ingredients.yeast',
       'isOrganic', 'isRetired', 'labels.contentAwareIcon',
       'labels.contentAwareLarge', 'labels.contentAwareMedium', 'labels.icon',
       'labels.large', 'labels.medium', 'name', 'nameDisplay',
       'originalGravity', 'servingTemperature', 'servingTemperatureDisplay',
       'socialAccounts', 'srm.hex', 'srm.id', 'srm.name', 'srmId', 'status',
       'statusDisplay', 'style.category.createDate', 'style.category.id',
       'style.category.name', 'style.categoryId', 'style.createDate',
       'style.description', 'style.id', 'style.name', 'style.shortName',
       'style.updateDate', 'styleId', 'updateDate', 'year'],
      dtype='object')

#### The same columns for s30 are available for the other 5, except that 'ingredients' was flattened

In [119]:
ipa_cat = pd.concat([concat26, concat31, concat171, concat172, concat173], axis=0, ignore_index=True)
ipa_cat.shape

(5124, 94)

In [120]:
ipa_cat = ipa_cat[['id', 'breweries', 'name','abv', 'description',
                   'ibu', 'ingredients.malt', 'ingredients.yeast', 
                   'ingredients.hops','socialAccounts', 'styleId']]

ipa_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5124 entries, 0 to 5123
Data columns (total 11 columns):
id                   5124 non-null object
breweries            5124 non-null object
name                 5124 non-null object
abv                  4935 non-null object
description          3740 non-null object
ibu                  2414 non-null object
ingredients.malt     81 non-null object
ingredients.yeast    19 non-null object
ingredients.hops     131 non-null object
socialAccounts       718 non-null object
styleId              5124 non-null int64
dtypes: int64(1), object(10)
memory usage: 440.4+ KB


#### Although the ingredients are only listed for about 3% of all the 14,774 IPA's in the 6 styles, I'm going to go through the trouble of unflattening the json_normalized ones in order to merge with the style 30 ones.

In [121]:
# first fillna to 0 for the NaN's
for col in ['ingredients.malt', 'ingredients.yeast', 'ingredients.hops']:
    ipa_cat[col] = ipa_cat[col].fillna(value=0)

In [127]:
# next define the transformation for each IPA row
def unflatten(row):
    result = dict()
    for ingredient in ['hops', 'yeast','malt']:
        if row[f'ingredients.{ingredient}']:
            result[ingredient] = row[f'ingredients.{ingredient}']
    return result

In [128]:
# apply the transformation to each row's ingredients
ipa_cat['ingredients'] = ipa_cat.apply(lambda row: unflatten(row), axis=1)

In [134]:
# check how many ingredients dicts are filled with something
sum(1 for notEmpty in ipa_cat.ingredients if notEmpty)

137

In [139]:
# remove the flat columns so you can concat with style 30
ipa_cat.drop(['ingredients.malt', 'ingredients.yeast', 'ingredients.hops'], axis=1, inplace=True)

#### Use similar tools to break socialAccounts apart and find a few untappd ID's hidden in the mix

In [57]:
# test on style 30 first
s30.socialAccounts = s30.socialAccounts.fillna(value=0)   

In [74]:
# The socialAccounts column occasionally has some links to uptappd beer IDs
import re
# There are some funky values here, not just int strings, so need regex
def get_handle(socialAccounts):
    '''Try to get untappd beer ID in case it exists and is needed to link.
    Apply this function after applying fillna to fill to zeros.
    '''
    if socialAccounts:
        for dic in socialAccounts:
            if 'link' in dic and 'untappd' in dic['link']:
                if 'handle' in dic:
                    try:  # getting ints from strings where possible
                        return int(re.findall(r'[0-9]+', dic['handle'])[0])
                    except IndexError:
                        return 0
    return 0
         

In [145]:
s30['untap'] = s30['socialAccounts'].apply(lambda x: get_handle(x))

In [146]:
sum(s30.untap > 0)

794

In [141]:
# now repeat for other 5 before concatting
ipa_cat.socialAccounts = ipa_cat.socialAccounts.fillna(value=0)
ipa_cat['untap'] = ipa_cat['socialAccounts'].apply(lambda x: get_handle(x))

In [150]:
# make sure cols line up before concatting
assert(len(ipa_cat.columns) == len(s30.columns))
[col for col in ipa_cat.columns if col not in s30.columns]

[]

In [151]:
ipa_cat = ipa_cat[s30.columns]

ipa_cat.columns == s30.columns

In [153]:
ipa_df = pd.concat([ipa_cat, s30], axis=0, ignore_index=True)

ipa_df.shape

In [155]:
ipa_df.drop('socialAccounts', axis=1, inplace=True)

ipa_df.head()

# come back later and unpack breweries and fillna some cols

In [157]:
ipa_df.to_csv('capstone_1/bdb_ipa_df.csv')